In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [2]:
def load():
    data = pd.read_csv("/kaggle/input/datasets/tboyle10/medicaltranscriptions/mtsamples.csv")
    return data

In [3]:
df = load()
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [4]:
#text cleaning
import nltk
import re
from nltk.corpus import stopwords

nltk.download("stopwords")
eng_stop_words = set(stopwords.words("english"))

def text_cleaning(text):
    text = str(text).lower()

    text = re.sub(r"\d+","",text)

    text = re.sub(r"[^\w\s]","",text)

    text = re.sub(r"\s+", " ", text).strip()

    clean_words = [
        word for word in text.split()
        if word not in eng_stop_words and len(word) > 2
    ]

    return " ".join(clean_words)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df["clean_transcription"] = df["transcription"].apply(text_cleaning)


In [6]:
labels = df["medical_specialty"]
labels

0              Allergy / Immunology
1                        Bariatrics
2                        Bariatrics
3        Cardiovascular / Pulmonary
4        Cardiovascular / Pulmonary
                   ...             
4994           Allergy / Immunology
4995           Allergy / Immunology
4996           Allergy / Immunology
4997           Allergy / Immunology
4998           Allergy / Immunology
Name: medical_specialty, Length: 4999, dtype: object

In [7]:
cleaned_transcription = df['clean_transcription'].dropna().tolist()
print(type(cleaned_transcription))
print(len(cleaned_transcription))
print(type(cleaned_transcription[0]))

<class 'list'>
4999
<class 'str'>


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# 1. Defining model 
bow_model = CountVectorizer(stop_words='english')

# Convert the first 75 transcriptions into numerical vectors
# Ensure 'cleaned_transcriptions' is a list of strings
vectorized_transcriptions = bow_model.fit_transform(cleaned_transcription[0:75])

# Get the vocabulary (feature names)
vocabulary = bow_model.get_feature_names_out()

# 4. Calculate frequency
word_counts = vectorized_transcriptions.sum(axis=0).A1

# 5. dict and the most commons
word_frequencies = dict(zip(vocabulary, word_counts))
top_5_most_common = Counter(word_frequencies).most_common(5)

print(top_5_most_common)

[('patient', np.int64(257)), ('right', np.int64(154)), ('bladder', np.int64(133)), ('left', np.int64(131)), ('placed', np.int64(126))]


#